## scratch

In [1]:
# coding: utf-8

import json
import base64
import requests
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import pymysql


class AESB64:
    def __init__(self, key):
        self.cipher = AES.new(key.encode('utf-8'), AES.MODE_ECB)

    def encrypt(self, cleartext):
        padded = pad(cleartext.encode('utf-8'), AES.block_size)
        encrypted = self.cipher.encrypt(padded)
        b64str = base64.b64encode(encrypted)
        return b64str

    def decrypt(self, b64str):
        encrypted = base64.b64decode(b64str)
        padded = self.cipher.decrypt(encrypted)
        cleartext = unpad(padded, AES.block_size).decode('utf-8')
        return cleartext
    

def fetch_api(config, all=True, pageSize=500):
    conn = config['connection']
    schema = config['schema']
    companies = config['companies']
    
    cipher = AESB64(conn['key'])
    
    api_data = {t.lower() + 'List': [] for t in schema['tables'].keys()}
    
    if all: # 全量数据
        urls = [conn['url.all']]
    else:   # 增量立案、结案
        urls = [conn['url.newLA'], conn['url.newJA']]
        
    for url in urls:
        if url == conn['url.newLA']:
            print "收集新增立案："
        elif url == conn['url.all']:
            print "收集全量案件："
        elif url == conn['url.newJA']:
            print "收集新增结案："
            
        for company in companies:
            pageNum = 1
            while True: # iterate over all pages
                request_payload = {
                    "loginName": conn['loginName'],
                    "passWord": conn['passWord'],
                    "company": company,
                    "pageNum": pageNum,
                    "pageSize": pageSize,
                    "flag": 1
                }

                response = requests.post(
                    url, 
                    data=cipher.encrypt(json.dumps(request_payload)),
                    headers={
                        "Content-Type": "application/json",
                        "ticket": conn['ticket']
                    }
                )

                if response.status_code != 200:
                    print "Request failed with status code {} for page {}.".format(response.status_code, pageNum)
                    break

                if response.text.encode('utf-8').startswith('{"msg":'):
                    print "Response: {}".format(response.text.encode('utf-8'))
                    pageNum += 1
                    continue
                    
                try:
                    decrypted_response = cipher.decrypt(response.text)
                except Exception as e:
                    print "Decryption failed for page {}".format(pageNum) 
                    break

                response_data = json.loads(decrypted_response)

                if "code" not in response_data or response_data["code"] != 1000:
                    print "Error in response: {}".format(response_data.get('msg', 'No message'))
                    print "Full response: {}".format(response_data)
                    break

                data = response_data.get("data")
                if not data:
                    print "No data field in response"
                    break

                hashMap = data.get("hashMap")
                if not hashMap:
                    print "No hashMap field in data"
                    break

                for table_name in api_data.keys():
                    table_data = hashMap.get(table_name)
                    if table_data:
                        api_data[table_name].extend(table_data)

                pages = data.get("pages", 0)
                if pages > 0:
                    print "{} 第 {}/{} 页".format(company.encode('utf-8'), pageNum, pages)

                if pageNum >= pages:
                    break

                pageNum += 1

    return api_data


def insert_db(api_data, config, batch_size=1000):
    conn = config['connection']
    schema = config['schema']

    list_to_table = {k.lower() + 'List': v for k, v in schema['tables'].items()}
    field_mappings = {schema['tables'][k]: v for k, v in schema['columns'].items()}

    connection = pymysql.connect(
        host=conn['db.host'],
        user=conn['db.user'],
        password=conn['db.password'],
        database=conn['db.database'],
        port=conn['db.port']
    )

    cursor = connection.cursor()
    
    for list_name, data in api_data.items():
        table_name = list_to_table.get(list_name)
        if not table_name:
            print "No table mapping for list {}.".format(list_name)
            continue

        field_mapping = field_mappings.get(table_name)
        if not field_mapping:
            print "No field mapping for table {}.".format(table_name)
            continue

        total_records = len(data)
        for i in range(0, total_records, batch_size):
            batch = data[i:i + batch_size]
            values_list = []

            for record in batch:
                values = [record.get(field) for field in field_mapping.keys()]
                values_list.append(values)

            columns = ', '.join(field_mapping.values())
            placeholders = ','.join(['%s'] * len(field_mapping))
            sql = "INSERT INTO {} ({}) VALUES ({})".format(table_name, columns, placeholders)

            try:
                cursor.executemany(sql, values_list)
            except Exception as e:
                print "Error inserting data into {}: {}".format(table_name, e)
                print "Batch start index: {}".format(i)

    connection.commit()
    cursor.close()
    connection.close()


In [4]:
import datetime

In [ ]:
datetime.time

In [2]:
with open("司法大数据配置.json", 'r') as config_json:
    config = json.load(config_json)  # 加载配置数据

In [5]:
data = fetch_api(config, all=False, pageSize=50)
insert_db(data, config)

收集新增立案：
中国五矿集团有限公司 第 1/48 页
中国五矿集团有限公司 第 2/48 页
中国五矿集团有限公司 第 3/48 页
中国五矿集团有限公司 第 4/48 页
中国五矿集团有限公司 第 5/48 页
中国五矿集团有限公司 第 6/48 页
中国五矿集团有限公司 第 7/48 页
中国五矿集团有限公司 第 8/48 页
中国五矿集团有限公司 第 9/48 页
中国五矿集团有限公司 第 10/48 页
中国五矿集团有限公司 第 11/48 页
中国五矿集团有限公司 第 12/48 页
中国五矿集团有限公司 第 13/48 页
中国五矿集团有限公司 第 14/48 页
中国五矿集团有限公司 第 15/48 页
中国五矿集团有限公司 第 16/48 页
中国五矿集团有限公司 第 17/48 页
中国五矿集团有限公司 第 18/48 页
中国五矿集团有限公司 第 19/48 页
中国五矿集团有限公司 第 20/48 页
中国五矿集团有限公司 第 21/48 页
中国五矿集团有限公司 第 22/48 页
中国五矿集团有限公司 第 23/48 页
中国五矿集团有限公司 第 24/48 页
中国五矿集团有限公司 第 25/48 页
中国五矿集团有限公司 第 26/48 页
中国五矿集团有限公司 第 27/48 页
中国五矿集团有限公司 第 28/48 页
中国五矿集团有限公司 第 29/48 页
中国五矿集团有限公司 第 30/48 页
中国五矿集团有限公司 第 31/48 页
中国五矿集团有限公司 第 32/48 页
中国五矿集团有限公司 第 33/48 页
中国五矿集团有限公司 第 34/48 页
中国五矿集团有限公司 第 35/48 页
中国五矿集团有限公司 第 36/48 页
中国五矿集团有限公司 第 37/48 页
中国五矿集团有限公司 第 38/48 页
中国五矿集团有限公司 第 39/48 页
中国五矿集团有限公司 第 40/48 页
中国五矿集团有限公司 第 41/48 页
中国五矿集团有限公司 第 42/48 页
中国五矿集团有限公司 第 43/48 页
中国五矿集团有限公司 第 44/48 页
中国五矿集团有限公司 第 45/48 页
中国五矿集团有限公司 第 46/48 页
中国五矿集团有限公司 第 47/48 页
中国五矿集团有限公司 第 4

In [52]:
len(data['msysList'])

4954

In [53]:
insert_db(data, config)

## original

In [2]:
import json
import base64
import requests
from Crypto.Cipher import AES
import pymysql

class AESCipher:
    def __init__(self, key):
        self.key = key.encode('utf-8')
        self.mode = AES.MODE_ECB

    def pad(self, data):
        block_size = AES.block_size
        padding = block_size - len(data) % block_size
        return data + (chr(padding) * padding).encode('utf-8')

    def unpad(self, data):
        return data[:-ord(data[-1])]

    def encrypt(self, raw):
        raw = self.pad(raw.encode('utf-8'))
        cipher = AES.new(self.key, self.mode)
        encrypted = base64.b64encode(cipher.encrypt(raw))
        return encrypted.decode('utf-8')

    def decrypt(self, enc):
        enc_bytes = base64.b64decode(enc)
        cipher = AES.new(self.key, self.mode)
        decrypted = self.unpad(cipher.decrypt(enc_bytes))
        return decrypted.decode('utf-8')


# Example usage
api_url = "https://qyss.cjbdi.com/qyss/qyss/enterlit/queryNewAllCase"
key = "wkjtC@ntdGscsoqc"
ticket = "1LyM07LLxWpTBFnTqHGMdp1vHqGTYBF9Ck289X7P8ionseKdd62lHH3gxvJPr0wx"
companies = ["五矿二十三冶建设集团有限公司", "中国一冶集团有限公司", "中国华冶科工集团有限公司",
             "长沙五矿商业管理有限公司"]
loginName = "zgwk2"
passWord = "shesu@2022&0198$"

all_data = {table: [] for table in [
    "msysList", "xzysList", "mszsList", "msesList", "xsesList",
    "sczxList", "xszsList", "xzzsList", "ccbqzxList", "xzesList",
    "hfzxList", "zxyyList", "xsysList"
]}
                  
# db_config = {
#     'host': '10.201.132.8',
#     'user': 'zhangyc6',
#     'password': 'zhangyc6@123456',
#     'database': 'dw_ods_dsep',
#     'port': 9030
# }

# insert_data_to_doris(all_data, db_config, mappings)

: 

In [7]:
aes_cipher = AESCipher(key)
pageNum = 1
pageSize = 100

headers = {
    "Content-Type": "application/json",
    "ticket": ticket
}
request_payload = {
    "loginName": loginName,
    "passWord": passWord,
    "company": companies[0],
    "pageNum": pageNum,
    "pageSize": pageSize,
    "flag": 1
}

encrypted_payload = aes_cipher.encrypt(json.dumps(request_payload))

response = requests.post(api_url, data=encrypted_payload, headers=headers)

assert response.status_code == 200, "Request failed with status code {}".format(response.status_code)

try:
    decrypted_response = aes_cipher.decrypt(response.text)
    print decrypted_response
except Exception as e:
    print "Decryption failed for page " + str(pageNum)
    print "Response text: " + response.text

type os encrypted_ascii is <type 'str'>
�qY��#wp���e'�������23��l��i�-��BW�Ƽ����*_3J�D/��淙t��S��.�{�X�)?'�43vɀ���g\���y\ ����\�g.��1��s�_Q|�������=g��ǧ��6;N��VP�*�}C���7�`��ە��0qo,����e�êXds��[�L�;��pn��<\��J!ͯ��J^X�����w�&�N`��]�Fu�|��/!m��L�����_�"���Ů ][��d/&*�,XT�g�����"����L��z�7�	�S0���65l?��S��eB[d�!��_g��H����5�|J�r� X
����G	.g��ә�.�VjK��J�\�25���["�/����+��	~ ��_��= ��zw�K&X��F1�z!��d7P�F�o�1���K��Pp��s�Ϻ�"u��Ocv4�B���~Ub!��/d1K�<��o����s]G��ǛI6�ea:�fY"Rr�g㳟zC�ML48�*�9X�?���[R�R�+�
�qY��#wp���e'�������23��l��i�-��BW�Ƽ����*_3J�D/��淙t��S��.�{�X�)6����w�J���\���y\ ����\�g.��1��s�_Q|�������=g��ǧ��6;N��VP�*�}C�`��&�e�"�1��o,����e�êXds��f�5u����u�rr
�T����'�/P�����ef�_��7�Y,�^j��)���p_���"��S��k�=�<U�#���Y���o����]��M'���{�c"	���Pɹ�-��tA�rZ�4cN-M�o
�������V"��с�R�{L��� �mhm�!�1~%�އ�{��T9Xl�V��@��Ǹc�����1翭�7��Ia-�Vs�D��U+�%���Yz����f$�<��ȝ���1������B��$�ѣ6"�rDXʆ��9��F��뙝WNX���